In [1]:
!kaggle datasets download -d  harishvutukuri/dogs-vs-wolves -p ../../data

dogs-vs-wolves.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
!unzip -q -u ../../data/dogs-vs-wolves.zip -d ../../data/dogs-vs-wolves

In [3]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile # TF temporal bugfix

In [4]:
%load_ext tensorboard

## Imports

In [5]:
import os

# Torch #
import torch
from torch import nn, optim
from torch.utils.data import DataLoader 

# Torchvision #
from torchvision import datasets, transforms, models

# Torchero #
import torchero
from torchero import SupervisedTrainer # Training the model
from torchero.callbacks.tensorboard import TensorBoardLogger
from torchero.callbacks import ProgbarLogger # Model supervising
from torchero.utils.data import train_test_split
from torchero.utils import show_imagegrid_dataset
from torchero.utils.tensorboard import SummaryWriter, write_imagegrid_dataset, write_model
from matplotlib import pyplot as plt

In [10]:
summary_writer = SummaryWriter('runs/Dogs vs Wolves')

## Training dataset

In [7]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

In [8]:
dataset = datasets.ImageFolder(root='../../data/dogs-vs-wolves/data', transform=transform)

In [9]:
train_ds, val_ds = train_test_split(dataset)

In [11]:
write_imagegrid_dataset(summary_writer, 'Training Dataset', train_ds)

In [12]:
%tensorboard --logdir runs/Dogs\ vs\ Wolves

### Test dataset

In [14]:
write_imagegrid_dataset(summary_writer, 'Test Dataset', val_ds)

### DataLoader

In [16]:
train_dl = DataLoader(train_ds, batch_size=10, num_workers=6, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=10, num_workers=6)

## Define model

In [17]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.weight.shape[-1], 1)
    
    def forward(self, x):
        x = self.model(x)
        x = x.squeeze()
        return x

In [18]:
model = Network()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /tmp/cache/torch/checkpoints/resnet18-5c106cde.pth


In [19]:
write_model(summary_writer, model, next(iter(train_dl))[0])

for param_group in trainer.optimizer.param_groups:
    print(param_group["lr"])

## Train model

In [20]:
trainer = SupervisedTrainer(model=model,
                            logging_frecuency=10,
                            optimizer='adam',
                            criterion='binary_cross_entropy_wl',
                            acc_meters=['recall_wl', 'precision_wl', 'binary_with_logits_accuracy', 'f1_wl'],
                            callbacks=[ProgbarLogger(notebook=True), TensorBoardLogger(summary_writer)])

In [21]:
trainer.cuda()

In [22]:
trainer.train(train_dl, val_dl, epochs=3)

summary_writer.close()

### Writting Embeddings to Projector

In [24]:
class Identity(nn.Module):
    def forward(self, x):
        return x

In [25]:
fc = model.model.fc.cpu()

In [26]:
model.model.fc = Identity()

In [27]:
from itertools import islice

In [28]:
trainer.cpu()

In [29]:
max_probas, predicted_classes = (torch.Tensor([[0.2,0.1,0.5],[0.4, 0.1, 0.2]])).topk(k=1)
max_probas = max_probas.squeeze(-1)
predicted_classes = predicted_classes.squeeze(-1)

In [30]:
predicted_classes

tensor([2, 0])

In [31]:
total_classes = []
embeddings = []
total_images = []
total_probas = []
predicted_classes = []
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((64, 64)), transforms.ToTensor()])
modes = []

dl = DataLoader(train_ds, batch_size=50, shuffle=True)

for ims, classes in islice(dl, 10):
    ims, classes = next(iter(dl))
    embs = model(ims)
    probas = torch.nn.functional.sigmoid(fc(embs)).squeeze(-1)
    predicted_classes.extend(probas > 0.5)
    total_probas.extend(probas)
    embeddings.append(embs)
    total_images.extend(transform(im) for im in ims)
    total_classes.extend(classes)
    modes.extend(['train']*len(classes))
    
dl = DataLoader(val_ds, batch_size=50, shuffle=True)

for ims, classes in islice(dl, 10):
    ims, classes = next(iter(dl))
    embs = model(ims)
    probas = torch.nn.functional.sigmoid(fc(embs)).squeeze(-1)
    predicted_classes.extend(probas > 0.5)
    total_probas.extend(probas)
    embeddings.append(embs)
    total_images.extend(transform(im) for im in ims)
    total_classes.extend(classes)
    modes.extend(['test']*len(classes))

/usr/lib/python3.8/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [32]:
embeddings = torch.stack(embeddings, 0)
embeddings = embeddings.reshape(-1, 512)
total_images = torch.stack(total_images, 0)

In [33]:
total_images.shape, len(total_classes), embeddings.shape

(torch.Size([900, 3, 64, 64]), 900, torch.Size([900, 512]))

In [34]:
summary_writer.add_embedding(embeddings.cpu(), metadata=[(dataset.classes[i], mode, dataset.classes[j], p.item(), 'ok' if i == j else 'fail') for mode, i, j, p in zip(modes, total_classes, predicted_classes, total_probas)], label_img=total_images, global_step=0, tag='embeddings', metadata_header=['label', 'mode', 'predicted_class', 'proba', 'correct'])

In [35]:
summary_writer.flush()

summary_writer.close()

In [36]:
from tensorboard import notebook

In [43]:
%tensorboard --logdir runs/Dogs\ vs\ Wolves --port 6609